In [1]:
import requests # 크롤링에 사용
from bs4 import BeautifulSoup # 크롤링에 사용

네이버 영화 사이트(https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date=20180101)

매월 1일 조회수 기준 TOP50 영화의 줄거리를 크롤링할 예정. 대상 기간은 2018.01~2021.02임.

1) 1차적으로 영화의 이름과 각 영화의 줄거리를 받아오기 위해 그 영화의 웹페이지로 들어갈 주소 크롤링

2) 크롤링한 주소를 이용하여 각 영화의 웹페이지에 이동, 거기서 줄거리를 크롤링

3) 데이터프레임으로 변환 후 csv 파일로 저장

4) 각 파일들을 데이터프레임으로 불러와서 모든 내용을 합친 후 저장
 

In [2]:
import re #정규표현식

def cleaning(x): #크롤링한 영화 줄거리 안에 있는 불피요한 문자를 제거하는 함수.
    text = "".join(map(str, x)) #리스트로 저장된 줄거리를 정규표현식 사용을 위해 문자열로 바꿔줌
    b = re.sub('\r',' ',text) #re.sub('처리할 내용', '대체할 내용', 대상 텍스트) 
    b = re.sub('<div class="story_area">','',b) # 때때로 \을 사용하여 문법문자->일반문자 변환에 주의
    b = re.sub('<div class="title_area">','',b)
    b = re.sub('<h4 class="h_story"><strong class="blind">줄거리</strong></h4>','',b)
    b = re.sub('href="http://terms.naver.com/ncrEntry.nhn\?dicId=moviework_dic&amp;ncrDocId=\D\D\d_\d\d\d"','',b)
    b = re.sub('<p class="con_tx">','',b)
    b = re.sub('<a class="movie_terms" ','',b)
    b = re.sub(' target="_blank">','',b)
    b = re.sub('<em class="blind">영화백과 보기</em></a>','',b)
    b = re.sub('<h5 class="h_tx_story">','',b)
    b = re.sub('\n','',b)
    b = re.sub('</div>','',b)
    b = re.sub('<br/>','',b)
    b = re.sub('\xa0','',b)
    b = re.sub('</p>','',b)
    b = re.sub('</h5>',' ',b)

    b = re.sub('<button class="story_more" ','',b)
    b = re.sub('id="toggleMakingnoteButton" ','',b)
    b = re.sub(' type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->','',b)
    b = re.sub('id="toggleMakingnoteButton" ','',b)
    b = re.sub('onclick="storyAndNote.toggleMakingnote\(\);"','',b)
    return b

In [3]:
import pandas as pd # 데이터프레임 사용
# 2018년
month = ["01","02","03","04","05","06","07","08","09","10","11","12"] #for문에서 사용

for i in month:
    url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date=2018"+i+"01" #for문의 i가 들어가며 url 자동 변경
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
    # headers를 사용하지 않으면 접속이 되지 않음
    
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') #뷰티풀수프 
    tem_name = [] #영화 이름을 담기 위해 임시로 사용할 리스트
    tem_link = [] #영화 링크를 담기 위해 임시로 사용할 리스트

    for link in soup.find_all('a'): # a 태그의 내용을 전부 가져옴
        tem_name.append(link.text.strip()) # 해당 부분의 text 내용만 tem_name에 추가
        tem_link.append(link.get('href')) # 영화 별 주소를 tem_link에 추가

    title = tem_name[32:82] # title을 출력해보면 32:82가 영화 랭킹에 해당, 각 영화의 제목
    address = tem_link[32:82] # tem_link을 출력해보면 32:82가 영화 랭킹에 해당, 각 영화 페이지의 주소 
    
    story = [] # 줄거리를 담을 리스트
    for j in range(len(title)): #len은 영화의 갯수(50)만큼
        url = "https://movie.naver.com"+address[j] #각 영화의 페이지에 접속하기 위해 사용
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
        response = requests.get(url, headers=headers)
        html = response.text
        soup1 = BeautifulSoup(html, 'html.parser')
        cont = soup1.find_all("div", {"class":"story_area"})
        story.append(cont)
        
    df = pd.DataFrame(title) #영화 제목을 이용하여 우선 데이터프레임 생성
    df1 = df.rename({0:'제목'},axis='columns') # 데이터프레임의 0열의 이름을 제목으로 변경 
    df1['주소 코드'] = address # '주소코드' 열을 추가
    df1['줄거리'] = story # '줄거리' 열을 추가

    df1['줄거리'] = df1['줄거리'].apply(cleaning) # 줄거리에서 불필요한 문자 제거
    
    df1.to_csv("./naver_movie1/2018"+i+".csv",header = None) #2018+월을 이름으로 하는 csv 파일 저장(12개), 헤더 비사용.

In [4]:
# 2019년 - 패턴은 2018년과 동일
month = ["01","02","03","04","05","06","07","08","09","10","11","12"]

for i in month:
    url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date=2019"+i+"01" #for문의 i가 들어가며 url 자동 변경
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') 
    tem_name = [] 
    tem_link = [] 

    for link in soup.find_all('a'): 
        tem_name.append(link.text.strip()) 
        tem_link.append(link.get('href')) 

    title = tem_name[32:82]
    address = tem_link[32:82]
    
    story = [] 
    for j in range(len(title)): 
        url = "https://movie.naver.com"+address[j] 
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
        response = requests.get(url, headers=headers)
        html = response.text
        soup1 = BeautifulSoup(html, 'html.parser')
        cont = soup1.find_all("div", {"class":"story_area"})
        story.append(cont)
        
    df = pd.DataFrame(title) 
    df1 = df.rename({0:'제목'},axis='columns') 
    df1['주소 코드'] = address 
    df1['줄거리'] = story 

    df1['줄거리'] = df1['줄거리'].apply(cleaning)
    
    df1.to_csv("./naver_movie1/2019"+i+".csv",header = None) 

In [5]:
# 2020년 - 패턴은 2018년과 동일하지만 11월은 해당 랭킹에 영화가 5개 밖에 들어있지 않아 에러가 나므로 제외
month = ["01","02","03","04","05","06","07","08","09","10","12"]

for i in month:
    url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date=2020"+i+"01" #for문의 i가 들어가며 url 자동 변경
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') 
    tem_name = [] 
    tem_link = [] 

    for link in soup.find_all('a'): 
        tem_name.append(link.text.strip()) 
        tem_link.append(link.get('href')) 

    title = tem_name[32:82]
    address = tem_link[32:82]
    
    story = [] 
    for j in range(len(title)): 
        url = "https://movie.naver.com"+address[j] 
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
        response = requests.get(url, headers=headers)
        html = response.text
        soup1 = BeautifulSoup(html, 'html.parser')
        cont = soup1.find_all("div", {"class":"story_area"})
        story.append(cont)
        
    df = pd.DataFrame(title) 
    df1 = df.rename({0:'제목'},axis='columns') 
    df1['주소 코드'] = address 
    df1['줄거리'] = story 

    df1['줄거리'] = df1['줄거리'].apply(cleaning)
    
    df1.to_csv("./naver_movie1/2020"+i+".csv",header = None) 

In [6]:
# 2021년 - 작업 당시 시점 2월까지 적용.
month = ["01","02"] 

for i in month:
    url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date=2021"+i+"01" #for문의 i가 들어가며 url 자동 변경
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') 
    tem_name = [] 
    tem_link = [] 

    for link in soup.find_all('a'): 
        tem_name.append(link.text.strip()) 
        tem_link.append(link.get('href')) 

    title = tem_name[32:82]
    address = tem_link[32:82]
    
    story = [] 
    for j in range(len(title)): 
        url = "https://movie.naver.com"+address[j] 
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
        response = requests.get(url, headers=headers)
        html = response.text
        soup1 = BeautifulSoup(html, 'html.parser')
        cont = soup1.find_all("div", {"class":"story_area"})
        story.append(cont)
        
    df = pd.DataFrame(title) 
    df1 = df.rename({0:'제목'},axis='columns') 
    df1['주소 코드'] = address 
    df1['줄거리'] = story 

    df1['줄거리'] = df1['줄거리'].apply(cleaning)
    
    df1.to_csv("./naver_movie1/2021"+i+".csv",header = None) 

In [7]:
# 갯수가 적은 2021년 파일로 우선 데이터 프레임 생성
df1 = pd.read_csv("./naver_movie1/202101.csv",header=None)
df2 = pd.read_csv("./naver_movie1/202102.csv",header=None)

df = pd.concat([df1,df2]) #데이터프레임 합침

df

,0,1,2,3
0,0,원더 우먼 1984,/movie/bi/mi/basic.nhn?code=163834,"세상이 기다린 히어로, 희망이 되다! 1984년 모든 것이 활기찬 시대, 다이애나 ..."
1,1,화양연화,/movie/bi/mi/basic.nhn?code=28876,화양연화花樣年華 가장 아름답고 찬란했던 시절 같은 날 같은 아파트로 이사 온 ‘첸...
2,2,도굴,/movie/bi/mi/basic.nhn?code=193194,“고물인 줄 알았는데 보물이었다?!”땅 파서 장사하는 도굴꾼들이 온다! 흙 맛만 봐...
3,3,조제,/movie/bi/mi/basic.nhn?code=199393,자신을 ‘조제’로 불러달라는 그녀 처음 만난 그날부터 ‘조제’는 ‘영석’에게 잊을 ...
4,4,이웃사촌,/movie/bi/mi/basic.nhn?code=164143,"적인가, 이웃인가?낮에는 친근한 이웃집 vs 밤에는 수상한 도청팀 백수가장 좌천위기..."
...,...,...,...,...
45,45,디노소어 어드벤쳐,/movie/bi/mi/basic.nhn?code=100152,"수석 연구원 ‘조나단’과 그의 조수와 의사, 승무원으로 형성된 영국 원정대는 공룡의..."
46,46,미션 파서블,/movie/bi/mi/basic.nhn?code=189124,"“돈 되니까” VS “국가를 위해” 우린, 한다면 한다! 입금만 되면 뭐든 가능해지..."
47,47,더블패티,/movie/bi/mi/basic.nhn?code=196361,세상을 향한 이들의 뒤집기 한.판.승! 입 찢어지게 햄버거를 먹던 너 냉삼에 소맥을...
48,48,언플랜드,/movie/bi/mi/basic.nhn?code=182525,미국 최대의 낙태 클리닉 가족계획연맹에서 8년간 상담사로 일하며최연소 소장에 오른 ...


In [8]:
#2018년
month = ["01","02","03","04","05","06","07","08","09","10","11","12"]

for i in month:
    tem = "df18_"+i
    tem = pd.read_csv("./naver_movie1/2018"+i+".csv",header=None) #헤더 비사용
    df = pd.concat([df,tem])
    
#2019년
month = ["01","02","03","04","05","06","07","08","09","10","11","12"]

for i in month:
    tem = "df19_"+i
    tem = pd.read_csv("./naver_movie1/2019"+i+".csv",header=None)
    df = pd.concat([df,tem])
    
#2020년 - 11월은 네이버 랭킹에 영화가 너무 적어 제외
month = ["01","02","03","04","05","06","07","08","09","10","12"]

for i in month:
    tem = "df20_"+i
    tem = pd.read_csv("./naver_movie1/2020"+i+".csv",header=None)
    df = pd.concat([df,tem])

In [9]:
df = df.reset_index() # 데이터 프레임을 합쳤지만 인덱스가 각 파일의 0-49로 돼있이므로 인덱스 초기화

In [10]:
df.drop(['index',0], 'columns') # 불필요한 열 삭제

,1,2,3
0,원더 우먼 1984,/movie/bi/mi/basic.nhn?code=163834,"세상이 기다린 히어로, 희망이 되다! 1984년 모든 것이 활기찬 시대, 다이애나 ..."
1,화양연화,/movie/bi/mi/basic.nhn?code=28876,화양연화花樣年華 가장 아름답고 찬란했던 시절 같은 날 같은 아파트로 이사 온 ‘첸...
2,도굴,/movie/bi/mi/basic.nhn?code=193194,“고물인 줄 알았는데 보물이었다?!”땅 파서 장사하는 도굴꾼들이 온다! 흙 맛만 봐...
3,조제,/movie/bi/mi/basic.nhn?code=199393,자신을 ‘조제’로 불러달라는 그녀 처음 만난 그날부터 ‘조제’는 ‘영석’에게 잊을 ...
4,이웃사촌,/movie/bi/mi/basic.nhn?code=164143,"적인가, 이웃인가?낮에는 친근한 이웃집 vs 밤에는 수상한 도청팀 백수가장 좌천위기..."
...,...,...,...
1845,힐빌리의 노래,/movie/bi/mi/basic.nhn?code=198754,미래가 걸린 중요한 일을 앞두고 고향으로 돌아갈 수밖에 없던 예일대 법대생이 가난하...
1846,마작,/movie/bi/mi/basic.nhn?code=31131,현대 도시인들의 처지와 시대에 대한 묘사를 통해 스스로 무엇을 원하는지도 알지 못하...
1847,소리도 없이,/movie/bi/mi/basic.nhn?code=187893,"악의 없이, 계획에 없던 유괴범이 되다! 범죄 조직의 하청을 받아 근면성실하고 전문..."
1848,파티마의 기적,/movie/bi/mi/basic.nhn?code=193195,"우리의 삶을 바꿀 기적의 영화! 1917년, 제1차 세계대전이 한창이던 어느 날 포..."


In [11]:
df.to_csv("./naver_movie1/naver_movie.csv",header = None) # csv로 파일 저장

느낀점

크롤링을 더 자동화 시키는 것에 대해서 생각해보기. 

자동화를 위해 도중에 발생하는 에러를 예외처리하는 것에 대해 생각 해보기

크롤링한 정보를 더 효과적으로 관리하는 방법에 대해 생각해보기

BeautifulSoup로 더 타겟 정보를 정확하게 좁혀서 수고를 줄일 수 있는 방법에 대해 생각해보기

크롤링의 대상 정보에 대해서 더 생각해보기

정규표현식을 이용하여 더 짧게 정제를 할 수 있는지 생각해보기
